In [23]:
import pandas as pd
import nltk
import re
import pickle

nltk.download('punkt')
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ukachi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ukachi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
airline_tweets = pd.read_csv("Tweets.csv")
airline_tweets["airline_sentiment"].nunique()

3

In [10]:
bipolar_tweets = airline_tweets[airline_tweets["airline_sentiment"] != "neutral"]

In [16]:
bipolar_tweets.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)
5,570300767074181121,negative,1.0000,Can't Tell,0.6842,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica seriously would pay $30 a fligh...,NaN,2015-02-24 11:14:33 -0800,NaN,Pacific Time (US & Canada)
6,570300616901320704,positive,0.6745,NaN,0.0000,Virgin America,NaN,cjmcginnis,NaN,0,"@VirginAmerica yes, nearly every time I fly VX...",NaN,2015-02-24 11:13:57 -0800,San Francisco CA,Pacific Time (US & Canada)


In [19]:
twitter_data = bipolar_tweets[["airline_sentiment", "text"]]

In [21]:
for col in twitter_data:
    print(twitter_data[col].nunique())

2
11381


In [22]:
twitter_data.isnull().sum()

airline_sentiment    0
text                 0
dtype: int64

### Now unto the pre-processing of the data

In [35]:
def preprocess(review):
    #convert the tweet to lower case
    review.lower()
    #convert all urls to sting "URL"
    review = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',review)
    #convert all @username to "AT_USER"
    review = re.sub('@[^\s]+','AT_USER', review)
    #correct all multiple white spaces to a single white space
    review = re.sub('[\s]+', ' ', review)
    #convert "#topic" to just "topic"
    review = re.sub(r'#([^\s]+)', r'\1', review)
    tokens = word_tokenize(review)
    tokens = [w for w in tokens if not w in stop_words]
    return " ".join(tokens)

def label_encode(sentiment):
    if sentiment == "positive":
        return 1
    elif sentiment == "neutral":
        return 1
    else:
        return 0
    

In [34]:
airline_tweets.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,review,sentiment
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada),NaN,NaN
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada),AT_USER plus 've added commercials experience ...,1.0
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada),NaN,NaN
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada),AT_USER 's really aggressive blast obnoxious `...,0.0
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada),AT_USER 's really big bad thing,0.0


In [37]:
airline_tweets["review"] = airline_tweets["text"].apply(preprocess)
airline_tweets["sentiment"] = airline_tweets["airline_sentiment"].apply(label_encode)

twitter_data = airline_tweets[["review", "sentiment"]]

twitter_data.head()

,review,sentiment
0,AT_USER What AT_USER said .,1
1,AT_USER plus 've added commercials experience ...,1
2,AT_USER I n't today ... Must mean I need take ...,1
3,AT_USER 's really aggressive blast obnoxious `...,0
4,AT_USER 's really big bad thing,0


In [38]:
twitter_data["sentiment"].value_counts()

0    9178
1    5462
Name: sentiment, dtype: int64

In [39]:
def save_file(filepath, data):
    """
    This function is used to save picklfiles
    Args:
        filepath: This is the location where it will be saved
        data: This is the data that you want to save
    Returns:
        None
    """
    pickle.dump(data, open(filepath, "wb"))

In [40]:
save_file("airline_twitter.pkl",twitter_data )